In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Importuemy potrzebne biblioteki**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import random

In [ ]:
snap=pd.read_csv('/kaggle/input/10k-snapchat-reviews/Snapchat_app_store_reviews.csv')

Przyjrzyjmy sie naszemu zbiorowi danych 

In [ ]:
snap.head(10)

In [ ]:
snap.info()

Sprawdzamy czy nie ma żadnych pustych rekordów

In [ ]:
pd.isnull(snap).sum()

Wykres pokazujący ilość poszczególnych ocen 

In [ ]:
sns.countplot(x='rating',data=snap)

In [ ]:
words=snap['review'].tolist()
words=''.join(words)
plt.figure(figsize=(12,12))
plt.imshow(WordCloud().generate(words))


Utworzenie kolumn sentiment i label oraz wypełnienie ich wartościami zależnie od wartości kolumny rating
Założyłem że ocena poniżej 4 jest oceną złą, jako że przeglądając kilka rekordów z ocena 3 są to raczej
recenzje zawierające jakies uwagi, problemy.

In [ ]:
snap['sentiment'] =''

In [ ]:
snap['label'] = np.nan
snap['label']= snap['rating'].apply(lambda x: 1 if (x > 3) else 0) 
snap

In [ ]:
snap['sentiment']= snap['rating'].apply(lambda x: 'good' if (x > 3) else 'bad') 

In [ ]:
snap

Utworzenie dwóch osobnych zbiorów positive/negative i utworzenie mapy słów dla poszczególnych zbiorów

In [ ]:
positive = snap.loc[(snap['sentiment'] == 'good')]
positive

In [ ]:
negative = snap.loc[(snap['sentiment'] == 'bad')]
negative

In [ ]:
words=positive['review'].tolist()
words=''.join(words)
plt.figure(figsize=(12,12))
plt.imshow(WordCloud().generate(words))

In [ ]:
words=negative['review'].tolist()
words=''.join(words)
plt.figure(figsize=(12,12))
plt.imshow(WordCloud().generate(words))

In [ ]:
words=snap['review'].tolist()
words=''.join(words)
plt.figure(figsize=(12,12))
plt.imshow(WordCloud().generate(words))

In [ ]:
snap["data"] = snap.apply(lambda x: str(x["title"]) + " " + str(x["review"]), axis=1)
snap["split"] = snap.apply(lambda x: "train" if random.randrange(0,100) > 10 else "valid", axis=1)

Wyciągnięcie uprzednio utworzonej kolumny label i przekształcenie jej w tablice numpy

In [ ]:
labels = pd.DataFrame(snap['label']).to_numpy()

In [ ]:
labels

Sprawdzenie wymiarów label

In [ ]:
labels.shape

Przyjrzenie się danym 

In [ ]:
snap['data'][0]

Przedstawione poniżej pętle odpowiadają kolejno za:
-usuwanie znaków innych niż litery
-usuwanie spacji
-zamiana na małe litery
-usunięcie stopwords czyli słów, które nie mają wpływu na znaczenie recenzji a tylko niepotrzebnie zanieczyszczają dane 
**Wzorowałem się tutaj na kodzie innego użytkownika kaggle**

In [ ]:
import re
snap1 = []
for i in range(len(snap['data'])):
    new=re.sub('[^a-zA-Z]', ' ', snap['data'][i])
    snap1.append(new)

In [ ]:
snap1[0]

In [ ]:
snap2 = []
for i in range(len(snap1)):
    new1=snap1[i].strip()
    snap2.append(new1)

In [ ]:
snap2[0]

In [ ]:
snap3 = []
for i in range(len(snap2)):
    new2=snap2[i].lower()
    snap3.append(new2)

In [ ]:
snap3[0]

In [ ]:
from nltk.corpus import stopwords
import string
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
name = ['snapchat', 'app', 'account', 'now', 'phone', 'snap', 'people']

snap4 = []

for i in snap3:
    sentence = i.split()
    s_p=[]
    for cor in sentence:
        if cor not in stop_words and cor not in punctuation and cor not in name:
            s_p.append(cor)
    s_p=" ".join(s_p)    
    snap4.append(s_p)


In [ ]:
snap4[0]

In [ ]:
texts=[]
for x in range(0,9559):
    texts.append(snap4[x])

Tutaj ustawiam długość recenzji, rozmiar próbek, korzystam z narzędzia tokenizer
Tworze zbiory treningowe i walidacyjne.
**Od tego momentu posiłkuje sie książką "Deep Learning. Praca z językiem Python i biblioteką Keras"**

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
maxlen = 200
training_samples = 500
validation_samples = 1000
max_words = 1000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Znaleziono %s unikatowych tokenów'%len(word_index))
data = pad_sequences(sequences,maxlen=maxlen)
labels = np.asarray(labels)
print(data.shape)
print(labels.shape)
indicies=np.arange(data.shape[0])
data = data[indicies]
labels=labels[indicies]
x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples:training_samples + validation_samples]
y_val = labels[training_samples:training_samples + validation_samples]

In [ ]:
glove_dir="/kaggle/input/glovedataset/"

embedding_index = {}
f = open(os.path.join(glove_dir,'glove.6B.100d.txt'),encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embedding_index[word] = coefs
f.close()
print('Found %s word vectors ' % len(embedding_index))

In [ ]:
embedding_dim = 100
embedding_matrix = np.zeros((max_words,embedding_dim))

for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            embedding_matrix[i]=embedding_vector
    

Utworzenie modelu. Jako że nasz model ma do czynienia tylko z dwoma klasami użyłem w ostatniej warstwie metody aktywacyjnej **sigmoid** oraz jako funkcje straty użyłem **binary crossentropy**

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding,Flatten,Dense,Dropout

model = Sequential()
model.add(Embedding(max_words,embedding_dim,input_length=maxlen))
model.add(Flatten())
model.add(Dense(32,activation='relu'))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.summary()

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding,Flatten,Dense,Dropout

model = Sequential()
model.add(Embedding(max_words,embedding_dim,input_length=maxlen))
model.add(Flatten())
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
history = model.fit(x_train,y_train,epochs=8,batch_size=32,validation_data=(x_val,y_val))

Sprawdzenie nazw kluczy i utworzenie wykresów pokazujących zależności pomiędzy stratą treningową i walidacji oraz dokładnością trenowania i walidacji 

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
accuracy = history.history['acc']
val_accuracy = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,len(accuracy)+1)
plt.plot(epochs,loss,'g',label='Strata treningowa')
plt.plot(epochs,val_loss,'b',label='Strata walidacji')
plt.title('Strata treningowa i walidacji')
plt.xlabel('Epoki')
plt.ylabel('Strata')
plt.legend()
plt.show()

In [ ]:
plt.clf()
plt.plot(epochs,accuracy,'g',label = 'Dokładność trenowania')
plt.plot(epochs,val_accuracy,'b',label = 'Dokładność walidacji')
plt.title('Dokładność trenowania i walidacji')
plt.xlabel('Epoki')
plt.ylabel('Strata')
plt.legend()
plt.show()

Jak widać model zaczyna ulegać nadmiernemu dopasowaniu. Aktualnie pracuje nad regularyzacją tego modelu

In [ ]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [ ]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
history = model.fit(x_train,y_train,epochs=10,batch_size=32,validation_data=(x_val,y_val))